# Extra opdracht - Functional Parsers
Een toepassing waar FP zich goed voor leent is het schrijven van parsers. Een parser is te beschrijven als een simpele en compositioneerbare functie. Een parser heeft het type $String \to [(a, String)]$: een input-string wordt ingelezen, en het resultaat is een lijst van mogelijke parses. Iedere parse bestaat op haar beurt weer uit een resultaat en een restant: het deel van de string dat (nog) niet nodig was. Hieronder zijn wat functies gedefiniëerd voor het combineren van parsers.
- `pure` maakt een parser die niets met de input doet en een meegegeven waarde als resultaat oplevert.
- `then` combineert twee parsers: beide parsers moeten op volgorde matchen, en de resultaten worden gecombineerd met een meegegeven functie.
- `either` combineert twee parsers door eerst de eerste te proberen, en als deze faalt de tweede te proberen. `either` geeft de mogelijkheid om meerdere alternatieven te combineren.
- `all` combineert een lijst parsers, op dezelfde manier als `then` twee parsers combineert. Alle parsers moeten hetzelfde type hebben, en de meegegeven functie wordt gebruikt om all resultaten te `fold`en.

In [ ]:
from typing import Callable, List, Tuple, TypeVar
import operator
from functools import reduce, partial
A = TypeVar('a')
B = TypeVar('b')
C = TypeVar('c')

def join(xss: List[List[A]]) -> List[A]:
    return reduce(lambda x,y: x+y, xss, [])

def pure(rv: A) -> Callable[[List[str]], List[Tuple[A, str]]]:
    """Lift a value into a parser; creates a parser that does not touch its input and returns a."""
    def parse(input: List[str]) -> List[Tuple[A,str]]:
        return [(rv, input)]
    return parse

def then(f: Callable[[A,B], C], \
        pa: Callable[[List[str]], List[Tuple[A, str]]], \
        pb: Callable[[List[str]], List[Tuple[B, str]]]) \
         -> Callable[[List[str]], List[Tuple[C, str]]]:
    """Chain two parsers together and combine the results using f."""
    def parse(input: List[str]) -> List[Tuple[C, str]]:
        return join(map(lambda r1: list(map(lambda r2: (f(r1[0], r2[0]), r2[1]), pb(r1[1]))), pa(input)))
    return parse

def either(pa: Callable[[List[str]], List[Tuple[A, str]]], \
           pb: Callable[[List[str]], List[Tuple[A, str]]]) \
            -> Callable[[List[str]], List[Tuple[A, str]]]:
    """Combine two parsers by taking the first one that succeeds."""
    def parse(input: List[str]) -> List[Tuple[A, str]]:
        r = pa(input)
        if len(r) > 0:
            return pa(input)
        else:
            return pb(input)
    return parse

def all(f: Callable[[A,A], A], \
       ps: List[Callable[[List[str]], List[Tuple[A, str]]]]) \
        -> Callable[[List[str]], List[Tuple[A, str]]]:
    """Chain a list of parsers together using f."""
    return(reduce(partial(then, f), ps))

def any(ps: List[Callable[[List[str]], List[Tuple[A, str]]]]) \
        -> Callable[[List[str]], List[Tuple[A, str]]]:
    """Choose the first succesfull parser from a list."""
    return(reduce(either, ps))

def repeat(p: Callable[[List[str]], List[Tuple[A, str]]]) \
           -> Callable[[List[str]], List[Tuple[List[A], str]]]:
    """Keep applying p until it fails."""
    def parse(input: List[str]) -> List[Tuple[List[A], str]]:
        if len(p(input)) > 0:
            return then(operator.add, p, repeat(p))(input)
        else:
            return pure("")(input)
    return parse

Hieronder zien we een voorbeeldparser, die een enkel gegeven karakter kan parsen. Parsers met een argument worden gecurry'd gebruikt: in dit geval wordt het argument `match` (het karakter dat gezocht wordt) eerst meegegeven, waarna het resultaat (technisch gezien een *parially applied function*) het standaard parser-type heeft. `parse_character` is een parser-generator, `parse_character('a')` is een voorbeeld van een parser die hiermee gemaakt kan worden. 

In dit geval heeft onze parser 0 of 1 mogelijke resultaten, dus een lege lijst of een singleton lijst. Andere parsers kunnen meerdere mogelijke resultaten hebben, zoals een parser die een reeks `a`'s matcht: deze kan 1, 2, ... `a`'s matchen, en al deze opties moeten beschikbaar zijn. Als de volgende parser bijvoorbeeld een enkele `a` nodig heeft kan deze slagen, waar dat met alleen de greedy optie niet zou kunnen.

De parser-generator `parse_string` matcht een langere string door meerdere `parse_character` instanties te combineren.

In [ ]:
def parse_character(match: str) -> Callable[[List[str]], List[Tuple[str, str]]]:
    """Parses a given character."""
    def parse(input: List[str]) -> List[Tuple[str, str]]:
        if len(input) == 0:              # Check for empty input
            return []
        else:
            head, *tail = input          # Split on first character
            if head == match:            # Check if this is the droid we are looking for
                return [(head, tail)]    # Return the match and the rest of the string.
            else:
                return []
    return parse

def parse_string(match: str) -> Callable[[List[str]], List[Tuple[str, str]]]:
    """Parses a given string."""
    return all(operator.add, map(parse_character, [*match]))

Hieronder zien we de parsers in actie. De test-string wordt eerst opgebroken in een lijst van karakters, zodat we geen verrassingen hebben over het type.

In [ ]:
# Example string
test = [*"Test"]

# Parse using explicit then on parse_character:
print(then(operator.add, either(parse_character("T"), parse_character("t")), 
           then(operator.add, parse_character("e"),
                then(operator.add, parse_character("s"),
                     parse_character("t"))))(test))

# Parse using all to combine a list of parse_character:
print(all(operator.add, [either(parse_character("T"), parse_character("t")), 
                         parse_character("e"),
                         parse_character("s"),
                         parse_character("t")])(test))

# Parse using parse_string:
print(parse_string("Test")(test))

# This should fail and return an empty list of results:
print(all(operator.add, [either(parse_character("E"), parse_character("e")), 
                         parse_character("r"),
                         parse_character("r"),
                         parse_character("o"),
                         parse_character("r")])(test))

## Rekenparsers
Met dit kleine opzetje mogen jullie zelf aan de slag. Je gaat een set parsers maken om simpele sommen te lezen en te berekenen. Schrijf hiertoe eerst een parser die een getal in kan lezen. Let erop dat een getal uit meerdere cijfers kan bestaan: de parser is pas klaar als ze een symbool tegenkomt dat geen cijfer is. Aan het type is te zien dat de parser een `int` terug gaat geven.

In [ ]:
def parse_number(input: List[str]) -> Callable[[List[str]], List[Tuple[int, str]]]:
    pass # Noot: als dit lukt door bouwblokken te chainen hoef je geen functienotatie te gebruiken;
         # parse_number = f(g(...)) is ook prima

print(parse_number([*"42"]))

### Operators
Nu kunnen we een parser maken die het plusteken herkent. Let ook hier weer op het type: we willen de plus niet als symbool opleveren, maar als een functie!

In [ ]:
def parse_plus(input: List[str]) -> Callable[[List[str]], List[Tuple[Callable[[int,int],int], str]]]:
    pass

print(parse_plus(["+"]))

### Applicatie

Door de twee hierboven gemaakte parsers te combineren kunnen we een optelling parsen. Parse een getal, negeer whitespace, parse een plusteken, negeer whitespace en parse een tweede getal. Het resultaat is de uitgerekende som.

In [ ]:
def ignoring_spaces(p: Callable[[str], List[Tuple[A, str]]]) \
                    -> Callable[[str], List[Tuple[A, str]]]:
    pass

def parse_sum(input: List[str]) -> Callable[[List[str]], List[Tuple[int, str]]]:
    pass

print(parse_sum([*"3+4"]))

Doe nu hetzelfde voor aftrekken, vermenigvuldigen, en delen. Abstraheer de `parse_plus` en `parse_sum` functies om voor de andere vier operatoren te werken. 

**NB:** Bij delen gaan we uit van `int` division, dus $5 / 2 = 2$. Definieer een algemene parser met behulp van `any`.

In [ ]:
def parse_operator(sym: str, \
                     f: Callable[[int,int],int]) \
                     -> Callable[[List[str]], List[Tuple[Callable[[int,int],int], str]]]:
    pass

def parse_operator_application(op: Callable[[List[str]], List[Tuple[Callable[[int,int],int], str]]]) \
                                -> Callable[[str], List[Tuple[int, str]]]:
    pass

# parse_difference, parse_product, parse_division

parse_math = None

print(parse_math([*"6/4"]))

### Haakjes
Nu gaan we proberen haakjes te parsen. Hiervoor moeten we de expressie tussen haakjes vinden en uitrekenen voordat het resultaat door wordt gegeven naar de grotere berekening. Probeer hier een parser voor te maken, en maak hierbij gebruik van `either` om de vier rekenkundige parsers aan te passen zodat deze ook een haakjes-expressie als argument kunnen gebruiken.

In [ ]:
# parse_parenthesised

print(parse_parenthesised([*"(1+2)"]))

### Negatieve getallen en putting it all together
Tot slot willen we ook negatieve getallen kunnen parsen. Als we dit slim aanpakken kunnen we negatieve haakjesexpressies ook meteen meepakken. Hiervoor hebben we een soort van hierarchie:
- Een getal of een haakjesxpressie is een `unsigned_expression`;
- Een minteken gevolgd door een `unsigned_expression` is een `signed_expression`;
- Een expression is `either` een `unsigned_expression` of een `signed_expression`;
- Plus, min, delen en vermenigvuldigen hebben `expression`s als parameters.

Maak een `parse_expression` die met complexe voorbeelden om kan gaan. Deze mag de volgorde van operaties negeren (dus $3 + 2 \times 7$ wordt $5 \times 7 = 35$ i.p.v. $3 + 14 = 17$), maar voor extra credit kan je hier ook naar kijken.

In [ ]:
# parse_unsigned_expression, parse_signed_expression, parse_expression

# Redefine any parsers using parse_number to use parse_expression

print(parse_expression([*"1 + (6/3) * -(5*2)"]))